# 🌞**Évaluation et Prévision des Énergies Renouvelables Variables (ERV)**

## 🎯 **Objectifs du Projet**

1. **Évaluer la production horaire des installations photovoltaïques** dans les régions de la France métropolitaine en utilisant des données climatiques et des observations des facteurs de capacité.

2. **Prédire la production énergétique des ERV à l'avance**, afin de mieux anticiper les variations de puissance et optimiser la gestion du réseau électrique.


coucouc c'est luc 

---